# 强化学习（Reinforcement learning，RL）

## 相关概念

![RL-1](./images/RL-1.png)

- Environment（环境）:包括**看得见的游戏界面**和**看不见的后台的程序**，负责产生游戏画面，根据你的动作切换游戏画面，生成敌人，奖励，决定何时结束游戏等。我们不能了解环境实现的细节，只能通过他设计的规则与他进行交互。

- Agent（智能体）：观察环境，根据自己的策略做出动作，争取获得更多的游戏得分。在玩游戏过程中，策略是由玩家提出的。**强化学习则让程序学出一个策略，可以得到更多的奖励。**

- State（状态）：游戏当前的状态。Agent 根据当前状态做出决策。

    - Observation：Observation 是 State 的一部分。在某些环境里，Agent 不能获得全部的视野，只能看到游戏的状态的一部分，成为 Observation。

- Action（动作）：Agent 做出的动作。在当前的 State 下，Agent 根据自己的策略决定作出的动作。

- Reward（奖励）：环境根据 Agent 的动作给出的奖励，Agent 最终要让自己的奖励最大化。Reward定义的好坏一定程度上会影响模型的训练。

- Action Space（动作可选择空间）：Agent 可以选择的动作的集合。马里奥的动作可选择空间包括{left, right, up}。

- Policy（策略函数）：输入 State，输出是 Action 的概率分布。一般用$\pi$表示。
  - $\pi(left|s_t) = 0.1 $
  - $\pi(right|s_t) = 0.2 $
  - $\pi(up|s_t) = 0.7 $
  - 在进行强化学习训练和推理是，一般不是执行概率最高的策略，而是根据概率分布进行采样。训练时要让模型探索各种动作的可能性，才能学到更好策略。推理时让输出多样性。

- Trajectory（轨迹）：一连串状态和动作的序列，也被称为 Episode，Rollout。一般用$\tau$表示。
  - {s_0, a_0, s_1, a_1, s_2, a_2, ...}：游戏初始状态为 $s_0$，Agent 做出动作 $a_0$；状态切换为 $s_1$，Agent 做出动作 $a_1$；以此类推，直到游戏结束。
  - Trajectory 中的下一个状态跟当前状态和当前动作有关。可以表示为：
    - $s_{t+1} = f(s_t, a_t)$ :下一个状态是确定的。
    - $s_{t+1} = f(·|s_t, a_t)$：下一个状态是随机的。比如抽卡

- Return（回报）：从当前时间点到游戏结束的 Reward 总和。我们希望最大化这个值。

## 数学概念：期望

老师告诉你，小明考试情况：
- 考80分概率为20%
- 考90分概率为80%

那么你对小明考试结果的预期（期望）为：
$0.2 * 80 + 0.8 * 90 = 88$

公式为：$ \sum_{x}^{} x\ast p(x)$

另一种办法，让小明考 n 次试，求平均：
$\frac{1}{n} \sum_{i=1}^{n} x_i$

期望：每个可能的结果的概率与其结果值的乘积之和。
$$
E(x)_{x\sim p(x)}  = \sum_{x}^{} x\ast p(x) \approx \frac{1}{n} \sum_{i=1}^{n} x_{x\sim p(x)}
$$
x 的期望 $E(x)$ 是根据 $p(x)$ 分布采样 $n$ 次的结果的平均值，当 $n$ 趋于无穷大时，右侧两个式子相等。


## RL 推导

**强化学习的目标则是训练一个 Policy 神经网络$\pi$，在所有状态$s$下，给出相应的 $a$（在所有可能的 $\tau$ 中），得到 Return 的期望最大。**
$$
E(R(\tau))_{\tau\sim P_{\theta } (\tau)} = \sum_{\tau}^{}R(\tau) P_{\theta } (\tau)
$$
其中，$\tau$ 是轨迹，$R(\tau)$ 是轨迹 $\tau$ 的 Return，$P_{\theta } (\tau)$ 是根据 Policy 神经网络 $\pi$ 得到的轨迹 $\tau$ 的概率，$\theta$ 是 Policy 神经网络 $\pi$ 的参数。


为了让期望最大，我们需要使用梯度上升的方法来更新 Policy 神经网络 $\pi$ 的参数 $\theta$。（也只能改变 $\theta$）：
$$
\nabla E(R(\tau))_{\tau\sim P_{\theta } (\tau)} = \nabla\sum_{\tau}^{}R(\tau) P_{\theta } (\tau) = \sum_{\tau}^{}R(\tau)\nabla P_{\theta } (\tau)
$$

将右式做数学变换，乘以一个$P_{\theta } (\tau)$再除以一个$P_{\theta } (\tau)$:
$$
\sum_{\tau}^{}R(\tau)\nabla P_{\theta } (\tau) = \sum_{\tau}^{}R(\tau)\nabla P_{\theta } (\tau)\frac{P_{\theta } (\tau)}{P_{\theta } (\tau)}
$$

将$P_{\theta } (\tau)$提出:
$$
\sum_{\tau}^{}R(\tau)\nabla P_{\theta } (\tau)\frac{P_{\theta } (\tau)}{P_{\theta } (\tau)} = \sum_{\tau}^{}P_{\theta } (\tau)R(\tau)\frac{\nabla P_{\theta } (\tau)}{P_{\theta } (\tau)}
$$

将$P_{\theta } (\tau)$视为取$\tau$的概率分布，$R(\tau)\frac{\nabla P_{\theta } (\tau)}{P_{\theta } (\tau)}$视为 $\tau$ 的奖励值。

根据期望的公式，可转化为求平均值的形式：
$$
\sum_{\tau}^{}P_{\theta } (\tau)R(\tau)\frac{\nabla P_{\theta } (\tau)}{P_{\theta } (\tau)}\approx \frac{1}{N}\sum_{n=1}^{N} R(\tau^{n})\frac{\nabla P_{\theta } (\tau^{n})}{P_{\theta } (\tau^{n})} 
$$

根据对数函数的求导公式$\nabla \log f(x) = \frac{\nabla f(x)}{f(x)}$，可得：
$$
\frac{1}{N}\sum_{n=1}^{N} R(\tau^{n})\frac{\nabla P_{\theta } (\tau^{n})}{P_{\theta } (\tau^{n})} = \frac{1}{N}\sum_{n=1}^{N} R(\tau^{n}) \nabla \log P_{\theta } (\tau^{n})
$$

已知，下一个状态$s_{t+1}$是由当前状态和当前动作决定的，那么当前的$\tau $的概率应该为从初始状态开始，多个状态和动作的乘积。
$$
P_{\theta } (\tau^{n}) = \prod_{t=1}^{T_n} P_{\theta }(a_{n}^{t}|s_{n}^{t})
$$

因此可推：
$$
\frac{1}{N}\sum_{n=1}^{N} R(\tau^{n}) \nabla \log P_{\theta } (\tau^{n}) =  \frac{1}{N}\sum_{n=1}^{N} R(\tau^{n}) \nabla \log \prod_{t=1}^{T_n} P_{\theta }(a_{n}^{t}|s_{n}^{t}) = \frac{1}{N}\sum_{n=1}^{N} R(\tau^{n}) \sum_{t=1}^{T_n} \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) = \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} R(\tau^{n})  \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 

该最终表达式可表示为对所有可能的$\tau$的期望最大的梯度。

通过梯度乘以学习率更新 Policy 神经网络的参数，即为 Policy gradient 算法（梯度策略算法）。

将求导符号去掉，得到：
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} R(\tau^{n}) \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$
即为我们要求得的使得期望最大的 Policy 神经网络的参数。

如果该$\tau$的 Return>0，那么就增大该$\tau$中所有状态下采取该动作的概率，如果 Return<0，那么就减小该$\tau$中所有状态下采取该动作的概率。

## 训练 Policy 神经网络

### 定义Loss函数

在需要最大化的Policy函数前面加上一个负号，即可得到Loss函数，让优化器最小化他。
$$
Loss = -\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} R(\tau^{n}) \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$

让神经网络连续**玩 n 场游戏**，得到 n 个 $\tau$，每个 $\tau$ 有一个 Return，即为 $R(\tau^{n})$。
![RL-3](./images/RL-3.png)


在每场游戏中，输入为当前的状态 $s_t$，进过神经网络后，通过 Softmax 函数，输出当前状态下通过 Agent 选取动作的概率分布。

随机采取一个概率值即为 $P_{\theta }(a_{n}^{t}|s_{n}^{t})$。
![RL-2](./images/RL-2.png)

**注意：选取动作并不是选取最大概率的一个，而是根据概率分布随机选取一个动作（随机采样）。**

通过这些值，进行训练，更新网络，流程为：
1. 玩游戏
2. 获得数据
3. 更新网络
4. 重复上述步骤

![RL-4](./images/RL-4.png)

这种更新策略称为 On Policy，采集数据和更新网络是同一个 Policy。

这会导致大部分时间都在采集数据，而不是在训练网络。

## 优化公式
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} R(\tau^{n})  \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$
如果该$\tau$的 Return>0，那么就增大该$\tau$中所有状态下采取该动作的概率，如果 Return<0，那么就减小该$\tau$中所有状态下采取该动作的概率。


问题一：
1. 是否增大或者减少动作的概率，应该关注在该动作之后到游戏结束时累积的 Reward，而不是整个游戏过程的 Reward。
2. 一个动作对接下 Reward 产生的影响应该是逐步衰减的，后续的 Reward 更多的是由当时的动作决定的。

基于以上两个问题，我们将 Return 优化为：
$$
R(\tau^{n}) = R(\tau^{n}) \longrightarrow \sum_{{t}'=t }^{T_n}\gamma ^{{t}'-t}r_{{t}'}^{n} = R_{t}^{n}
$$
转为从当前步 t 到游戏结束时累计的 Reward。其中$\gamma$是一个衰减因子，其值在(0,1)之间，通过指数函数来衰减对后续 Reward 的影响。

得到公式：
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} R_{t}^{n} \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$

问题二：

此外，在好的局势和坏的局势下，无论采取什么动作，都会得到正的 Reward 或者负的 Reward。这时，算法就会增加或减少所有的动作的概率，突出的 Reward 增加或减少的大一些，但整体上训练会变慢。

![RL-5](./images/RL-5.png)

为此，我们为每个 Reward 都减去一个 baseline，baseline 如果选择合适，无论是在好的局势还是坏的局势下，都能得到有正有负的 Reward。

让相对好的概率增加，相对差的概率减少，使训练加快。

将公式加入 baseline $B(s_{t}^{n})$，得到：
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} (R_{t}^{n} - B(s_{t}^{n})) \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$
$B(s_{t}^{n})$ 也需要使用神经网络进行训练，这便是 Actor-Critic 算法。

做动作的是 Actor，对 Actor 打分的是 Critic。该算法要训练 Actor 和 Critic 两个网络。

问题三：

已知，$R_{t}^{n}$是具体的随机采样，随机采得的样本方差很大，会导致训练不稳定。

为此，我们引入一个动作价值函数（Action-Value Function），记为 $Q_{\theta }(s, a)$，表示在 s 状态下，选择动作 a 期望得到的回报。

引入一个状态价值函数（State-Value Function），记为 $V_{\theta }(s)$，表示在 s 状态下，无论做什么动作，期望得到的回报。

将两个函数相减，得到**优势函数**（Advantage Function），记为：
$$
A_{\theta }(s, a)=Q_{\theta }(s, a)-V_{\theta }(s)
$$
表示在 s 状态下，选择动作 a 比其他动作能带来多少优势。

可以看到，baseline 也是一个优势函数，公式被推导为：
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta }(s_{n}^{t}, a_{n}^{t}) \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$


### 计算优势函数

在动作价值函数中，在 t 时刻做出动作 a，期望得到的 Return 值为当前得到的 Reward 和后续带来的影响：
$$
Q_{\theta }(s_t, a)=r_t + \gamma * V_{\theta }(s_{t+1})
$$
这也是行为价值函数和状态价值函数的关系，代入优势函数后得：
$$
A_{\theta }(s_t, a)=r_t + \gamma * V_{\theta }(s_{t+1}) - V_{\theta }(s_t)
$$
优势函数里只剩状态价值函数。

针对下一步的状态价值函数$V_{\theta }(s_{t+1})$，可以将其粗略的表示为下一步得到的 Reward 和后续带来的影响：
$$
V_{\theta }(s_{t+1}) \approx r_{t+1} + \gamma * V_{\theta }(s_{t+2})
$$
依次类推，可以对下 X 步进行采样：

$ A_{\theta }^{1}(s_t, a)=r_t + \gamma * V_{\theta }(s_{t+1}) - V_{\theta }(s_t) $

$ A_{\theta }^{2}(s_t, a)=r_t + \gamma * r_{t+1} + \gamma^2 * V_{\theta }(s_{t+2}) - V_{\theta }(s_t) $

$ A_{\theta }^{3}(s_t, a)=r_t + \gamma * r_{t+1} + \gamma^2 * r_{t+2} + \gamma^3 * V_{\theta }(s_{t+3}) - V_{\theta }(s_t) $

......

$ A_{\theta }^{T}(s_t, a)=r_t + \gamma * r_{t+1} + \gamma^2 * r_{t+2} + \gamma^3 * r_{t+3} + ··· + \gamma^T * V_{\theta }(s_{t+T}) - V_{\theta }(s_t) $

定义一个中间变量$\delta _{t}^{v}$，表示在第 t 步执行动作带来的优势，使得：

$
\delta _{t}^{v} = r_t + \gamma * V_{\theta }(s_{t+1}) - V_{\theta }(s_t)
$

$
\delta _{t+1}^{v} = r_t+1 + \gamma * V_{\theta }(s_{t+2}) - V_{\theta }(s_t+1)
$

将 $\delta$ 带入到 X 步优势函数中：

$ A_{\theta }^{1}(s_t, a)= \delta _{t}^{v} $

$ A_{\theta }^{2}(s_t, a)= \delta _{t}^{v} + \gamma * \delta _{t+1}^{v} $

$ A_{\theta }^{3}(s_t, a)= \delta _{t}^{v} + \gamma * \delta _{t+1}^{v} + \gamma^2 * \delta _{t+2}^{v} $

······

采样的步数越多，具体的 reward 越多，使得方差越大，偏差越小（情况更多，更接近真实值）。

采样步数越少，更多的 reward 是被状态价值函数估算得出，使得方差越小（不会碰到极值），偏差越大。

那么该采样几步呢？

## Generalized Advantage Estimation (GAE)

在 GAE 函数中，依次进行一步采样，两步采样，三步采样，直到 X 步，并分配不同的权重，将其加和表示为最终的优势函数：

$$
A_{\theta }^{GAE} = (1-\lambda )(A_{\theta }^{1} + \lambda * A_{\theta }^{2} + \lambda^{2} * A_{\theta }^{3} + ...)
$$


如果$\lambda = 0.9$，那么优势函数为：
$
A_{\theta }^{GAE} = 0.01 * A_{\theta }^{1} + 0.09 * A_{\theta }^{2} + 0.081 * A_{\theta }^{3} + ...
$

将$\delta$ 代入到 GAE 函数的$A$中：

$A_{\theta }^{GAE}(s_{t},a) = (1-\lambda )(A_{\theta }^{1} + \lambda * A_{\theta }^{2} + \lambda^{2} * A_{\theta }^{3} + ...)$

$= (1-\lambda )(\delta _{t}^{v}+ \lambda * (\delta _{t}^{v} + \gamma * \delta _{t+1}^{v}) + \lambda^{2} * (\delta _{t}^{v} + \gamma * \delta _{t+1}^{v} + \gamma^2 * \delta _{t+2}^{v}) + ...) $

$= (1-\lambda )(\delta _{t}^{v}*(1+ \lambda + \lambda^2 +  ...)+ \gamma * \delta _{t+1}^{v}*(\lambda + \lambda^2 +  ...)+...)$  可以看到有一个**等比数列**，整理得到：

$ = (1-\lambda )(\delta _{t}^{v}\frac{1}{1-\lambda } + \gamma * \delta _{t+1}^{v}\frac{1}{1-\lambda }  + ...)$

$ = \delta _{t}^{v} + \gamma * \lambda * \delta _{t+1}^{v} + ...$

$ = \sum_{b=0}^{\infty}(\gamma \lambda )^{b}\delta _{t+b}^{V}$

最后的得到 GAE 优势函数为：
$$
A_{\theta }^{GAE}(s_{t},a)=\sum_{b=0}^{\infty}(\gamma \lambda )^{b}\delta _{t+b}^{V}
$$
$$
\delta _{t}^{v} = r_t + \gamma * V_{\theta }(s_{t+1}) - V_{\theta }(s_t)
$$

带入策略梯度优化目标函数得：
$$
\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta }^{GAE}(s_{n}^{t},a_{n}^{t}) \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) 
$$
使其越大越好。

其中，我们将状态价值函数使用神经网络拟合。

一般，状态价值函数与策略函数共用同一参数，只是最后一层不同，通过一个神经元输出当前状态的价值。
![RL-6](./images/RL-6.png)

$$
Label = \sum_{t'=t}^{T_n}\gamma^{t'-t}r_{t'}^{n} 
$$

该公式表示统计当从前步到结束的**所有reward的衰减加和**作为 Label，衰减系数为 $\gamma$。

# 参考
- https://www.bilibili.com/video/BV1iz421h7gb/?spm_id_from=333.337.search-card.all.click&vd_source=8fb10652f8c3316e5308e66bcf6011f0
